In [80]:
pip install torch torchvision torchaudio torchtext --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [81]:
pip install portalocker>=2.0.0

Note: you may need to restart the kernel to use updated packages.


In [82]:
import torch
import torchtext
import os
import collections

os.makedirs('./data', exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

In [83]:
# check the paltform, Apple Silicon or Linux
import os, platform

torch_device = 'mps' if platform.system() == 'Darwin' else 'cpu'

In [84]:
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [85]:
torch_device

'cpu'

In [86]:
print(train_dataset)

ShardingFilterIterDataPipe


In [87]:
for i,x in zip(range(5), train_dataset):
    print(f'**{classes[x[0]]}** -> {x[1]}\n')

**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.

**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.

**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.

**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.

**Sci/Tech** -> Oil prices soa

In [88]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

In [89]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
first_sentence = train_dataset[0][1]
second_sentence = train_dataset[1][1]

f_tokens = tokenizer(first_sentence)
s_tokens = tokenizer(second_sentence)

print(f'\nfirst token list: \n{f_tokens}')
print(f'\nsecond token list: \n{s_tokens}')


first token list: 
['wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '(', 'reuters', ')', 'reuters', '-', 'short-sellers', ',', 'wall', 'street', "'", 's', 'dwindling\\band', 'of', 'ultra-cynics', ',', 'are', 'seeing', 'green', 'again', '.']

second token list: 
['carlyle', 'looks', 'toward', 'commercial', 'aerospace', '(', 'reuters', ')', 'reuters', '-', 'private', 'investment', 'firm', 'carlyle', 'group', ',', '\\which', 'has', 'a', 'reputation', 'for', 'making', 'well-timed', 'and', 'occasionally\\controversial', 'plays', 'in', 'the', 'defense', 'industry', ',', 'has', 'quietly', 'placed\\its', 'bets', 'on', 'another', 'part', 'of', 'the', 'market', '.']


In [90]:
import torchtext

counter =  collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))

vocab = torchtext.vocab.vocab(counter, min_freq=1)

In [91]:
word_lookup_f = [list((vocab[w],w) for w in f_tokens)]
print(f'\nIndex lockup in 1st sentence:\n{word_lookup_f}')

word_lookup_s = [list((vocab[w],w) for w in s_tokens)]
print(f'\nIndex lockup in 2nd sentence:\n{word_lookup_s}')


Index lockup in 1st sentence:
[[(0, 'wall'), (1, 'st'), (2, '.'), (3, 'bears'), (4, 'claw'), (5, 'back'), (6, 'into'), (7, 'the'), (8, 'black'), (9, '('), (10, 'reuters'), (11, ')'), (10, 'reuters'), (12, '-'), (13, 'short-sellers'), (14, ','), (0, 'wall'), (15, 'street'), (16, "'"), (17, 's'), (18, 'dwindling\\band'), (19, 'of'), (20, 'ultra-cynics'), (14, ','), (21, 'are'), (22, 'seeing'), (23, 'green'), (24, 'again'), (2, '.')]]

Index lockup in 2nd sentence:
[[(25, 'carlyle'), (26, 'looks'), (27, 'toward'), (28, 'commercial'), (29, 'aerospace'), (9, '('), (10, 'reuters'), (11, ')'), (10, 'reuters'), (12, '-'), (30, 'private'), (31, 'investment'), (32, 'firm'), (25, 'carlyle'), (33, 'group'), (14, ','), (34, '\\which'), (35, 'has'), (36, 'a'), (37, 'reputation'), (38, 'for'), (39, 'making'), (40, 'well-timed'), (41, 'and'), (42, 'occasionally\\controversial'), (43, 'plays'), (44, 'in'), (7, 'the'), (45, 'defense'), (46, 'industry'), (14, ','), (35, 'has'), (47, 'quietly'), (48, 'pl

In [92]:
vocab_size = len(vocab)
print(f'Vocab size: {vocab_size}')

def encode(text):
    return [vocab.get_stoi()[s] for s in tokenizer(text)]

vec=encode(train_dataset[0][1])
print(vec)

Vocab size: 95810
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 10, 12, 13, 14, 0, 15, 16, 17, 18, 19, 20, 14, 21, 22, 23, 24, 2]


In [93]:
def decode(text):
    return [vocab.get_itos()[i] for i in text]

print(decode(vec))

['wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '(', 'reuters', ')', 'reuters', '-', 'short-sellers', ',', 'wall', 'street', "'", 's', 'dwindling\\band', 'of', 'ultra-cynics', ',', 'are', 'seeing', 'green', 'again', '.']


In [94]:
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer()
# corpus = [
#     'I like hot dogs.',
#     'The dog ran fast.',
#     'Its hot outside.',
# ]

# vectorizer.fit_transform(corpus)
# vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

In [95]:
vocab_size = len(vocab)

def to_bow(text, bow_vocab_size=vocab_size):
    """
    Convert text string to a bag-of-words tensor.
    """
    res = torch.zeros(bow_vocab_size, dtype=torch.float32)
    for i in encode(text):
        if i <bow_vocab_size:
            res[i] += 1
    return res

print(f'sample text:{train_dataset[0][1]}')
print(f'bow vector: {to_bow(train_dataset[0][1])}')

sample text:Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
bow vector: tensor([2., 1., 2.,  ..., 0., 0., 0.])


In [105]:

net = torch.nn.Sequential(
    torch.nn.Linear(vocab_size, 4),
    torch.nn.LogSoftmax(dim=1)
)

net=net.to(torch_device)


In [106]:
def train_eopch(net, dataloader, lr=0.01, optimizer=None, loss_fn=torch.nn.NLLLoss(), epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr=lr)
    net.train()
    total_loss, acc, count,i = 0,0,0,0
    for labels, features in dataloader:
        labels, features = labels.to(torch_device), features.to(torch_device)
        optimizer.zero_grad()
        output = net(features)
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss
        _, predicted = torch.max(output, 1)
        acc += (predicted == labels).sum()
        count += len(labels)
        i+=1
        if i%report_freq == 0:
            print(f'iteration {count}, loss {total_loss.item()/count}, accuracy {acc.item()/count}') # item() is used to get the value of a tensor
        if epoch_size  and count >= epoch_size:
            break
    
    return total_loss.item()/count, acc.item()/count

train_eopch(net, train_loader, epoch_size=1)

(0.08671943843364716, 0.1875)

In [107]:
bigram_vectorizer = CountVectorizer(ngram_range=(1,2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
    'I like hot dogs.',
    'The dog ran fast.',
    'Its hot outside.',
]

bigram_vectorizer.fit_transform(corpus)
print("Vocabulary: ", bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

Vocabulary:  {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [108]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l, ngrams=2))

bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print(f'Bigram vocabulary length: {len(bi_vocab)}')

Bigram vocabulary length: 1308842
